<center><h1> Truss Investment </h1></center>
<center><h1> Trades Data Analysis Report</h1></center>
  
### May 2021 
---

In [ ]:
##### LIBRARIES AND FUNCTIONS#####

# Load libraries
import pandas as pd
import ipywidgets as widgets
from ipywidgets import Layout, interact, interactive
from IPython.display import clear_output, display, HTML
import datetime as dt
from datetime import datetime, date
import plotly.express as px
import plotly.graph_objects as go

# Import functions
from functions import *
      
# Display options 
pd.set_option('display.max_rows', 500) # all rows
pd.options.mode.chained_assignment = None # spress warning
pd.set_option('display.float_format', lambda x: '%.3f' % x)

In [ ]:
##### READ DATASETS #####

# Get trades dataset list
dfs_list = Get_Trades()

# Read BR receipt data
BR_receipt = pd.read_csv('BR Receipt.csv')

# Read US Receipt data
US_futures_receipt = pd.read_csv('US Futures Receipt.csv')
US_equities_receipt = pd.read_csv('US Equities Receipt.csv')

# Read all trades receipt data
all_trades = pd.read_csv('All Trades Receipt.csv')

In [ ]:
##### DATE PICKER #####

# Get start and end date
today = date.today()
begin_date = dt.datetime(2020, 7, 1).date()

# Create begin date picker
begin_period = widgets.DatePicker(description='Start:',
                                 value = begin_date)

# Create end date picker
end_period = widgets.DatePicker(description='End:',
                               value = today)

In [ ]:
# Function to get data for BR_trades
def Trade_Select(trade):
    
    for i in range(0, len(dfs_list)):
        
        if (dfs_list[i]['Ticker'][0] == trade):
            
            return(dfs_list[i])

In [ ]:
##### BRAZIL TRADES DROPDOWN MENU #####

# Create dropdown menu to selct trade
BR_trade_dropdown = interactive(Trade_Select,
                                trade = widgets.Combobox(
                                    options= ['DOLM21',
                                              'DOLK21',
                                              'DOLG21',
                                              'DOLQ20',
                                              'DOLU20',
                                              'INDG21',
                                              'INDJ21',
                                              'INDM21',
                                              'ISPH21',
                                              'ISPM21',
                                              'ISPU20',
                                              'ISPU21',
                                              'ISPZ20',
                                              'WDOF21',
                                              'WDOG21',
                                              'WDOJ21',
                                              'WDOK21',
                                              'WDOM21',
                                              'WDOQ20',
                                              'WDOU20',
                                              'WING21',
                                              'WINJ21',
                                              'WINM21',
                                              'WSPH21',
                                              'WSPM21',
                                              'WSPU20',
                                              'WSPZ20'],
                                    description="Trade:")
                               )

In [ ]:
##### BRAZIL TRADES OPERATIONAL CHECKBOX #####

# Create vector with operations
data = ["Buy Quant", "Buy Price", "Sell Quant", "Sell Price", "Px Close"]

# Create checkbox operations
BR_checkboxes_operation = [widgets.Checkbox(value=True, indent=False, description=label) for label in data]

# Create checkbox operations horizontal box
BR_hbox_operation = widgets.HBox(children=BR_checkboxes_operation, layout=Layout(width='600px'))

In [ ]:
##### BRAZIL TRADES FINANCIAL CHECKBOX #####

# Create vector with financial
data = ["Hold Pos", "MtoM P&L", "Gross P&L", "Comm", "Exc Fee", "Net P&L"]

# Create checkbox financial
BR_checkboxes_financial = [widgets.Checkbox(value=True, indent=False, description=label) for label in data]

# Create checkbox financial horizontal box
BR_hbox_financial = widgets.HBox(children=BR_checkboxes_financial, layout=Layout(width='700px'))

In [ ]:
##### BRAZIL TRADES BUTTONS #####

# Create button and output
BR_trades_button = Button_Sumbmit()

# Create button output
BR_trades_output = widgets.Output()

# Create button and output
BR_trades_clean = Button_Clean()

# Create button output
BR_trades_output_clean = widgets.Output()

# Unite buttons
BR_trades_ui = widgets.HBox([BR_trades_button, BR_trades_clean])

In [ ]:
##### BRAZIL TRADES BUTTONS FUNCTIONS #####

# Create button clicked function
def BR_Trades_Clicked(b):  
    with BR_trades_output:
        
        # Clean output
        clear_output()
        
        # Condition if no trade is selected
        if (BR_trade_dropdown.result is None):
            
            # Display warning message box
            prRed('#################################### ERROR - PLEASE SELECT A TRADE! ####################################')
       
        else:
        
            # Create selected_data list
            selected_data = []
        
            # Fixede variables
            fixed = ['Date', 'Ticker']
        
            # Add fixed variables
            selected_data = selected_data + fixed
                    
            # Loop operation filter
            for i in range(0, len(BR_checkboxes_operation)):
                    if BR_checkboxes_operation[i].value == True:
                        selected_data = selected_data + [BR_checkboxes_operation[i].description]
        
            # Loop financial filter
            for j in range(0, len(BR_checkboxes_financial)):
                    if BR_checkboxes_financial[j].value == True:
                        selected_data = selected_data + [BR_checkboxes_financial[j].description]
                    
            # Create trade dataframe
            df = BR_trade_dropdown.result
            
            # Round values
            df = df.round({'Px Close':2, 'Sell Price':2, 'Buy Price': 2, 'Exc Fee':2, 'MtoM P&L':2, 'Gross P&L': 2, 'Net P&L': 2, 'Comm': 2})
        
            # Change columns type
            df = df.astype({'Sell Quant': int, 'Buy Quant': int, 'Hold Pos': int})
        
            # Filter data
            df = df[selected_data] 
               
            # Transform into date type
            df['Date'] = pd.to_datetime(df['Date'], format='%d/%m/%Y')
        
            # Apply filter
            df = df[(df['Date'] >= pd.Timestamp(begin_period.value)) &  (df['Date'] <= pd.Timestamp(end_period.value))]   
        
            # Sort by date
            df = df.sort_values(by = 'Date')
        
            # Refine date
            df['Date'] = df['Date'].dt.strftime('%d/%m/%Y')

            # Reset index
            df = df.reset_index(drop = True)
        
            #Create red columns vector
            red_col = ['MtoM P&L' , 'Gross P&L', 'Net P&L', 'Hold Pos']
       
            # Create auxiliar variable
            aux = []
        
            # Loop to define red columns
            for i in range(0,len(red_col)):
                if(red_col[i] in df.columns):
                    aux.append(red_col[i])
                    
            if aux == []:
                
                # Print result (without red columns)
                display(df.style.set_precision(2))
                
            else:
            
                # Print result (with red columns)
                display(df.style.applymap(color_negative_red, subset=pd.IndexSlice[:, aux]).set_precision(2))
            
# Create clean button function
def BR_Trades_Clean_Clicked(b):  
    with BR_trades_output_clean:
        
        # Clean output
        BR_trades_output.clear_output()

In [ ]:
##### BRAZIL TRADES TABLE ON BUTTON CLIKED #####

# On button click
BR_trades_button.on_click(BR_Trades_Clicked)
BR_trades_clean.on_click(BR_Trades_Clean_Clicked)

In [ ]:
##### BRAZIL TRADES DROPDOWN MENU #####

# Create dropdown menu to selct trade
BR_rolagem_dropdown = interactive(Trade_Select,
                                trade = widgets.Combobox(
                                    options= ['DR1K21M21',
                                             'RSPH21M21',
                                             'WS1H21M21'],
                                    description="Rolagem:")
                               )

In [ ]:
##### BRAZIL TRADES ROLAGEM ROLAGEM OPERATIONAL CHECKBOX #####

# Create vector with operations
data = ["Buy Quant", "Sell Quant", 'Buyer Buy Price', 'Buyer Sell Price', 'Seller Buy Price', "Seller Sell Price"]

# Create checkbox operations
BR_rolagem_checkboxes_operation = [widgets.Checkbox(value=True, indent=False, description=label) for label in data]

# Create checkbox operations horizontal box
BR_rolagem_hbox_operation = widgets.HBox(children=BR_rolagem_checkboxes_operation, layout=Layout(width='900px'))

In [ ]:
##### BRAZIL TRADES ROLAGEM FINANCIAL CHECKBOX #####

# Create vector with financial
data = ["Buyer Gross P&L", "Seller Gross P&L"]

# Create checkbox financial
BR_rolagem_checkboxes_financial = [widgets.Checkbox(value=True, indent=False, description=label) for label in data]

# Create checkbox financial horizontal box
BR_rolagem_hbox_financial = widgets.HBox(children=BR_rolagem_checkboxes_financial, layout=Layout(width='300px'))

In [ ]:
##### BRAZIL TRADES ROLAGEM BUTTONS #####

# Create button and output
BR_rolagem_button = Button_Sumbmit()

# Create button output
BR_rolagem_output = widgets.Output()

# Create button and output
BR_rolagem_clean = Button_Clean()

# Create button output
BR_rolagem_output_clean = widgets.Output()

# Unite buttons
BR_rolagem_ui = widgets.HBox([BR_rolagem_button, BR_rolagem_clean])

In [ ]:
##### BRAZIL TRADES ROLAGEM BUTTONS FUNCTIONS #####

# Create button clicked function
def BR_Rolagem_Clicked(b):  
    with BR_rolagem_output:
        
        # Clean output
        clear_output()
        
        # Condition if no trade is selected
        if (BR_rolagem_dropdown.result is None):
            
            # Display warning message box
            prRed('#################################### ERROR - PLEASE SELECT A ROLAGEM! ####################################')
       
        else:
        
            # Create selected_data list
            selected_data = []
        
            # Fixede variables
            fixed = ['Date', 'Buyer Ticker', 'Seller Ticker']
        
            # Add fixed variables
            selected_data = selected_data + fixed
                    
            # Loop operation filter
            for i in range(0, len(BR_rolagem_checkboxes_operation)):
                    if BR_rolagem_checkboxes_operation[i].value == True:
                        selected_data = selected_data + [BR_rolagem_checkboxes_operation[i].description]
        
            # Loop financial filter
            for j in range(0, len(BR_rolagem_checkboxes_financial)):
                    if BR_rolagem_checkboxes_financial[j].value == True:
                        selected_data = selected_data + [BR_rolagem_checkboxes_financial[j].description]
                    
            # Create trade dataframe
            df = BR_rolagem_dropdown.result
            
            # Round values
            df = df.round({'Buyer Buy Price': 2, 'Buyer Sell Price': 2 , 'Seller Buy Price': 2 , "Seller Sell Price": 2, "Buyer Gross P&L": 2, "Seller Gross P&L": 2})
        
            # Change columns type
            df = df.astype({'Sell Quant': int, 'Buy Quant': int})
        
            # Filter data
            df = df[selected_data] 
               
            # Transform into date type
            df['Date'] = pd.to_datetime(df['Date'], format='%d/%m/%Y')
        
            # Apply filter
            df = df[(df['Date'] >= pd.Timestamp(begin_period.value)) &  (df['Date'] <= pd.Timestamp(end_period.value))]   
        
            # Sort by date
            df = df.sort_values(by = 'Date')
        
            # Refine date
            df['Date'] = df['Date'].dt.strftime('%d/%m/%Y')

            # Reset index
            df = df.reset_index(drop = True)
        
            #Create red columns vector
            red_col = ['Buyer Gross P&L', 'Seller Gross P&L']
       
            # Create auxiliar variable
            aux = []
        
            # Loop to define red columns
            for i in range(0,len(red_col)):
                if(red_col[i] in df.columns):
                    aux.append(red_col[i])
                    
            if aux == []:
                
                # Print result (without red columns)
                display(df.style.set_precision(2))
                
            else:
            
                # Print result (with red columns)
                display(df.style.applymap(color_negative_red, subset=pd.IndexSlice[:, aux]).set_precision(2))
            
# Create clean button function
def BR_Rolagem_Clean_Clicked(b):  
    with BR_rolagem_output_clean:
        
        # Clean output
        BR_rolagem_output.clear_output()

In [ ]:
##### BRAZIL TRADES ROLAGEM ON BUTTON CLIKED #####

# On button click
BR_rolagem_button.on_click(BR_Rolagem_Clicked)
BR_rolagem_clean.on_click(BR_Rolagem_Clean_Clicked)

In [ ]:
###### BRAZIL TRADES VISUALIZATION BUTTONS #####

# Create button and output
BR_trades_viz_button = Button_Sumbmit()

# Create button output
BR_trades_viz_output = widgets.Output()

# Create button and output
BR_trades_viz_clean = Button_Clean()

# Create button output
BR_trades_viz_output_clean = widgets.Output()

# Unite buttons
BR_trades_viz_ui = widgets.HBox([BR_trades_viz_button, BR_trades_viz_clean])

In [ ]:
##### BRAZIL TRADES VISUALIZATION BUTTONS FUNCTIONS #####

# Create button clicked function
def BR_Trades_Viz_Clicked(b):  
    with BR_trades_viz_output:
        
        # Clean output
        clear_output()
        
        # Condition if no trade is selected
        if (BR_trade_dropdown.result is None):
            
            # Display warning message box
            prRed('#################################### ERROR - PLEASE SELECT A TRADE! ####################################')
       
        else:
         
            # Create trade dataframe
            df = BR_trade_dropdown.result

            # Transform into date type
            df['Date'] = pd.to_datetime(df['Date'], format='%d/%m/%Y')

            # Apply filter
            df = df[(df['Date'] >= pd.Timestamp(begin_period.value)) &  (df['Date'] <= pd.Timestamp(end_period.value))]   

            # Transform df format
            df_plot = pd.melt(df[['Buy Quant', 'Sell Quant']])

            # Concat Date
            df_plot['Date'] = pd.concat([df['Date'], df['Date']]).reset_index(drop = True)

            # Concat Px Close Diff
            df_plot['Px Close Diff'] = pd.concat([abs(df['Px Close'].diff()), abs(df['Px Close'].diff())]).reset_index(drop = True)

            # Plot graph
            fig = px.bar(df_plot, x= "Date", y= "value", color= "variable", title= 'The plot shows trade ') 

            # Update fig layout
            fig.update_layout(barmode= 'group')
            fig.update_yaxes(title='')

            # Add close price difference
            fig.add_trace(go.Scatter(mode= "lines", x= df_plot["Date"], y= df_plot["Px Close Diff"], name= "Close Variance"))

            # Plot graph
            fig.show()
        
# Create clean button function
def BR_Trades_Viz_Clean_Clicked(b):  
    with BR_trades_viz_output_clean:
        
        # Clean output
        BR_trades_viz_output.clear_output()

In [ ]:
##### BRAZIL TRADES VISUALIZATION ON BUTTON CLIKED #####

# On button click
BR_trades_viz_button.on_click(BR_Trades_Viz_Clicked)
BR_trades_viz_clean.on_click(BR_Trades_Viz_Clean_Clicked)

In [ ]:
##### BRAZIL RECEIPT CHECKBOX #####

# Create vector with checkbox data
data = ['Receipt Value', 'FX Description', 'FX Value', 'FX Rate', 'Comm', 'Exc Fee', 'Reg Fee']

# Create checkbox receipt
BR_receipt_checkboxes = [widgets.Checkbox(value=True, indent=False, description=label) for label in data]

# Create checkbox financial horizontal box
BR_receipt_hbox = widgets.HBox(children=BR_receipt_checkboxes, layout=Layout(width='900px'))

In [ ]:
##### BRAZIL RECEIPT CUMSUM CHECKBOX #####

# Create vector with cumsum
data = ['Receipt Value Cum Sum', 'FX Value Cum Sum', 'Total Cum Sum', 'Settlement Cum Sum']

# Create checkbox cumsum
BR_receipt_checkboxes_cumsum = [widgets.Checkbox(value=True, indent=False, description=label) for label in data]

# Create checkbox cumsum horizontal box
BR_receipt_hbox_cumsum = widgets.HBox(children=BR_receipt_checkboxes_cumsum, layout=Layout(width='700px'))

In [ ]:
##### BRAZIL RECEIPT BUTTONS #####

# Create button and output
BR_receipt_button = Button_Sumbmit()

# Create button output
BR_receipt_output = widgets.Output()

# Create button and output
BR_receipt_clean = Button_Clean()

# Create button output
BR_receipt_output_clean = widgets.Output()

# Unite buttons
BR_receipt_ui = widgets.HBox([BR_receipt_button, BR_receipt_clean])

In [ ]:
##### BRAZIL RECEIPT BUTTONS FUNCTIONS #####

# Create button clicked function
def BR_Receipt_Clicked(b):
    with BR_receipt_output:
        
        # Clean output
        clear_output()
        
        # Create receipt dataframe
        df = BR_receipt
        
        # Create selected_data list
        selected_data = []
        
        # Add fixed variables
        selected_data = selected_data + ['Date']
                    
        # Loop operation filter
        for i in range(0, len(BR_receipt_checkboxes)):
                if BR_receipt_checkboxes[i].value == True:
                    selected_data = selected_data + [BR_receipt_checkboxes[i].description]
                    
        # Loop operation filter
        for j in range(0, len(BR_receipt_checkboxes_cumsum)):
                if BR_receipt_checkboxes_cumsum[j].value == True:
                    selected_data = selected_data + [BR_receipt_checkboxes_cumsum[j].description]
                    
        # Filter selected data
        df = df[selected_data]
  
        # Transform into date type
        df['Date'] = pd.to_datetime(df['Date'], format='%d/%m/%Y')
        
        # Apply filter
        df = df[(df['Date'] >=  pd.Timestamp(begin_period.value)) & (df['Date'] <= pd.Timestamp(end_period.value))] 
        
        # Sort by date
        df = df.sort_values(by = 'Date')
        
        # Refine date
        df['Date'] = df['Date'].dt.strftime('%d/%m/%Y')
        
        # Reset index
        df = df.reset_index(drop =True)
        
        # Red column vector
        red_col = ['Receipt Value', 'FX Value', 'Receipt Value Cum Sum', 'FX Value Cum Sum', 'Settlement Cum Sum']
        
        # Create auxiliar variable
        aux = []
        
        # Loop to define red columns
        for i in range(0,len(red_col)):
            if(red_col[i] in df.columns):
                aux.append(red_col[i])
        if aux == []:
            
            # Print result (without red columns)
            display(df.style.set_precision(2))
            
        else:
            
            # Print result (with red columns)
            display(df.style.applymap(color_negative_red, subset=pd.IndexSlice[:, aux]).set_precision(2))
        
# Create button clicked function
def BR_Receipt_Clean_Clicked(b):
    with BR_receipt_output_clean:
        
        # Clear output_receipt
        BR_receipt_output.clear_output()

In [ ]:
##### BRAZIL RECEIPT ON BUTTON CLIKED #####

# On button click
BR_receipt_button.on_click(BR_Receipt_Clicked)
BR_receipt_clean.on_click(BR_Receipt_Clean_Clicked)

In [ ]:
##### US FUTURES TRADES DROPDOWN MENU #####

# Create dropdown menu to selct trade
US_futures_dropdown = interactive(Trade_Select,
                                trade = widgets.Combobox(
                                    options= ['6L F21',
                                             '6L G21',
                                             '6L H21',
                                             '6L J21',
                                             '6L K21',
                                             '6L M21',
                                             '6L Q20',
                                             '6L U20',
                                             '6L V20',
                                             '6L X20',
                                             '6L Z20',
                                             'ES H21',
                                             'ES M21',
                                             'ES U20',
                                             'ES Z20',
                                             'MES H21',
                                             'MES M21',
                                             'MES U20',
                                             'MES Z20'],
                                    description="Trade:"))

In [ ]:
##### US FUTURES TRADES OPERATIONAL CHECKBOX #####

# Create vector with operations
data = ["Buy Quant", "Buy Price", "Sell Quant", "Sell Price", "Px Close"]

# Create checkbox operations
US_futures_trades_checkboxes_operation = [widgets.Checkbox(value=True, indent=False, description=label) for label in data]

# Create checkbox operations horizontal box
US_futures_trades_hbox_operation = widgets.HBox(children=US_futures_trades_checkboxes_operation, layout=Layout(width='600px'))

In [ ]:
##### US FUTURES TRADES FINANCIAL CHECKBOX #####

# Create vector with financial
data = ["Hold Pos", "MtoM P&L", "Gross P&L", "Comm", "Exc Fee", "Net P&L"]

# Create checkbox financial
US_futures_trades_checkboxes_financial = [widgets.Checkbox(value=True, indent=False, description=label) for label in data]

# Create checkbox financial horizontal box
US_futures_trades_hbox_financial = widgets.HBox(children=US_futures_trades_checkboxes_financial, layout=Layout(width='700px'))

In [ ]:
##### US FUTURES TRADES BUTTONS #####

# Create button and output
US_futures_trades_button = Button_Sumbmit()

# Create button output
US_futures_trades_output = widgets.Output()

# Create button and output
US_futures_trades_clean = Button_Clean()

# Create button output
US_futures_trades_output_clean = widgets.Output()

# Unite buttons
US_futures_trades_ui = widgets.HBox([US_futures_trades_button, US_futures_trades_clean])

In [ ]:
##### US FUTURES TRADES BUTTONS FUNCTIONS #####

# Create button clicked function
def US_Futures_Trades_Clicked(b):
    with US_futures_trades_output:
        
        # Clean output
        clear_output()
        
        # Condition if no trade is selected
        if (US_futures_dropdown.result is None):
            
            # Display warning message box
            prRed('#################################### ERROR - PLEASE SELECT A TRADE! ####################################')
        
        else:
            # Create selected_data list
            selected_data = []

            # Fixede variables
            fixed = ['Date', 'Ticker']

            # Add fixed variables
            selected_data = selected_data + fixed

            # Loop operation filter
            for i in range(0, len(US_futures_trades_checkboxes_operation)):
                    if US_futures_trades_checkboxes_operation[i].value == True:
                        selected_data = selected_data + [US_futures_trades_checkboxes_operation[i].description]

            # Loop financial filter
            for j in range(0, len(US_futures_trades_checkboxes_financial)):
                    if US_futures_trades_checkboxes_financial[j].value == True:
                        selected_data = selected_data + [US_futures_trades_checkboxes_financial[j].description]

            # Create trade dataframe
            df = US_futures_dropdown.result

            # Round values
            df = df.round({'Px Close':2, 'Sell Price':2, 'Buy Price': 2, 'Exc Fee':2, 'MtoM P&L':2, 'Gross P&L': 2, 'Net P&L': 2, 'Comm': 2})

            # Change columns type
            df = df.astype({'Sell Quant': int, 'Buy Quant': int, 'Hold Pos': int})

            # Filter data
            df = df[selected_data] 

            # Transform into date type
            df['Date'] = pd.to_datetime(df['Date'], format='%d/%m/%Y')

            # Apply filter
            df = df[(df['Date'] >=  pd.Timestamp(begin_period.value)) & (df['Date'] <= pd.Timestamp(end_period.value))] 

            # Sort by date
            df = df.sort_values(by = 'Date')

            # Refine date
            df['Date'] = df['Date'].dt.strftime('%d/%m/%Y')

            # Reset index
            df = df.reset_index(drop =True)

            #Create red columns vector
            red_col = ['MtoM P&L' , 'Gross P&L', 'Net P&L', 'Hold Pos']

            # Create auxiliar variable
            aux = []

            # Loop to define red columns
            for i in range(0,len(red_col)):
                if(red_col[i] in df.columns):
                    aux.append(red_col[i])
            if aux == []:

                # Print result (without red columns)
                display(df.style.set_precision(2))

            else:

                # Print result (with red columns)
                display(df.style.applymap(color_negative_red, subset=pd.IndexSlice[:, aux]).set_precision(2))
        
# Create button clicked function
def US_Futures_Trades_Clean_Clicked(b):
    with US_futures_trades_output_clean:
        
        # Clear output_receipt
        US_futures_trades_output.clear_output()

In [ ]:
##### US FUTURES TRADES ON BUTTON CLIKED #####

# On button click
US_futures_trades_button.on_click(US_Futures_Trades_Clicked)
US_futures_trades_clean.on_click(US_Futures_Trades_Clean_Clicked)

In [ ]:
##### US EQUITIES TRADES DROPDOWN MENU #####

# Create dropdown menu to selct trade
US_equities_dropdown = interactive(Trade_Select,
                                trade = widgets.Combobox(
                                    options= ['MS',
                                             'MS Options',
                                             'PBR',
                                             'SOXX 430',
                                             'SOXX 410'],
                                    description="Trade:"))

In [ ]:
##### US EQUITIES TRADES OPERATIONAL CHECKBOX #####

# Create vector with operations
data = ["Buy Quant", "Buy Price", "Sell Quant", "Sell Price", "Px Close"]

# Create checkbox operations
US_equities_trades_checkboxes_operation = [widgets.Checkbox(value=True, indent=False, description=label) for label in data]

# Create checkbox operations horizontal box
US_equities_trades_hbox_operation = widgets.HBox(children=US_equities_trades_checkboxes_operation, layout=Layout(width='600px'))

In [ ]:
##### US EQUITIES TRADES FINANCIAL CHECKBOX #####

# Create vector with financial
data = ["Hold Pos", "MtoM P&L", "Gross P&L", "Comm", "Exc Fee", "Net P&L"]

# Create checkbox financial
US_equities_trades_checkboxes_financial = [widgets.Checkbox(value=True, indent=False, description=label) for label in data]

# Create checkbox financial horizontal box
US_equities_trades_hbox_financial = widgets.HBox(children=US_equities_trades_checkboxes_financial, layout=Layout(width='700px'))

In [ ]:
##### US EQUITIES TRADES BUTTONS #####

# Create button and output
US_equities_trades_button = Button_Sumbmit()

# Create button output
US_equities_trades_output = widgets.Output()

# Create button and output
US_equities_trades_clean = Button_Clean()

# Create button output
US_equities_trades_output_clean = widgets.Output()

# Unite buttons
US_equities_trades_ui = widgets.HBox([US_equities_trades_button, US_equities_trades_clean])

In [ ]:
##### US EQUITIES TRADES BUTTONS FUNCTIONS #####

# Create button clicked function
def US_Equities_Trades_Clicked(b):
    with US_equities_trades_output:
        
        # Clean output
        clear_output()
        
        # Condition if no trade is selected
        if (US_equities_dropdown.result is None):
            
            # Display warning message box
            prRed('#################################### ERROR - PLEASE SELECT A TRADE! ####################################')
        
        else:
            # Create selected_data list
            selected_data = []

            # Fixede variables
            fixed = ['Date', 'Ticker']

            # Add fixed variables
            selected_data = selected_data + fixed

            # Loop operation filter
            for i in range(0, len(US_equities_trades_checkboxes_operation)):
                    if US_equities_trades_checkboxes_operation[i].value == True:
                        selected_data = selected_data + [US_equities_trades_checkboxes_operation[i].description]

            # Loop financial filter
            for j in range(0, len(US_equities_trades_checkboxes_financial)):
                    if US_equities_trades_checkboxes_financial[j].value == True:
                        selected_data = selected_data + [US_equities_trades_checkboxes_financial[j].description]

            # Create trade dataframe
            df = US_equities_dropdown.result

            # Round values
            df = df.round({'Px Close':2, 'Sell Price':2, 'Buy Price': 2, 'Exc Fee':2, 'MtoM P&L':2, 'Gross P&L': 2, 'Net P&L': 2, 'Comm': 2})

            # Change columns type
            df = df.astype({'Sell Quant': int, 'Buy Quant': int, 'Hold Pos': int})

            # Filter data
            df = df[selected_data] 

            # Transform into date type
            df['Date'] = pd.to_datetime(df['Date'], format='%d/%m/%Y')

            # Apply filter
            df = df[(df['Date'] >=  pd.Timestamp(begin_period.value)) & (df['Date'] <= pd.Timestamp(end_period.value))] 

            # Sort by date
            df = df.sort_values(by = 'Date')

            # Refine date
            df['Date'] = df['Date'].dt.strftime('%d/%m/%Y')

            # Reset index
            df = df.reset_index(drop =True)

            #Create red columns vector
            red_col = ['MtoM P&L' , 'Gross P&L', 'Net P&L', 'Hold Pos']

            # Create auxiliar variable
            aux = []

            # Loop to define red columns
            for i in range(0,len(red_col)):
                if(red_col[i] in df.columns):
                    aux.append(red_col[i])
            if aux == []:

                # Print result (without red columns)
                display(df.style.set_precision(2))

            else:

                # Print result (with red columns)
                display(df.style.applymap(color_negative_red, subset=pd.IndexSlice[:, aux]).set_precision(2))
        
# Create button clicked function
def US_Equities_Trades_Clean_Clicked(b):
    with US_equities_trades_output_clean:
        
        # Clear output_receipt
        US_equities_trades_output.clear_output()

In [ ]:
##### US EQUITIES TRADES ON BUTTON CLIKED #####

# On button click
US_equities_trades_button.on_click(US_Equities_Trades_Clicked)
US_equities_trades_clean.on_click(US_Equities_Trades_Clean_Clicked)

In [ ]:
###### US TRADES VISUALIZATION BUTTONS #####

# Create button and output
US_trades_viz_button = Button_Sumbmit()

# Create button output
US_trades_viz_output = widgets.Output()

# Create button and output
US_trades_viz_clean = Button_Clean()

# Create button output
US_trades_viz_output_clean = widgets.Output()

# Unite buttons
US_trades_viz_ui = widgets.HBox([US_trades_viz_button, US_trades_viz_clean])

In [ ]:
##### US TRADES VISUALIZATION BUTTONS FUNCTIONS #####

# Create button clicked function
def US_Trades_Viz_Clicked(b):  
    with US_trades_viz_output:
        
        # Clean output
        clear_output()
        
        # Condition if no trade is selected
        if (US_futures_dropdown.result is None):
            
            # Display warning message box
            prRed('#################################### ERROR - PLEASE SELECT A TRADE! ####################################')
        
        else:
            
            # Create trade dataframe
            df = US_futures_dropdown.result

            # Transform into date type
            df['Date'] = pd.to_datetime(df['Date'], format='%d/%m/%Y')

            # Apply filter
            df = df[(df['Date'] >= pd.Timestamp(begin_period.value)) &  (df['Date'] <= pd.Timestamp(end_period.value))]   

            # Transform df format
            df_plot = pd.melt(df[['Buy Quant', 'Sell Quant']])

            # Concat Date
            df_plot['Date'] = pd.concat([df['Date'], df['Date']]).reset_index(drop = True)

            # Concat Px Close Diff
            df_plot['Px Close Diff'] = pd.concat([abs(df['Px Close'].diff()), abs(df['Px Close'].diff())]).reset_index(drop = True)

            # Plot graph
            fig = px.bar(df_plot, x= "Date", y= "value", color= "variable", title= 'The plot shows trade ') 

            # Update fig layout
            fig.update_layout(barmode= 'group')
            fig.update_yaxes(title='')

            # Add close price difference
            fig.add_trace(go.Scatter(mode= "lines", x= df_plot["Date"], y= df_plot["Px Close Diff"], name= "Close Variance"))

            # Plot graph
            fig.show()
        
# Create clean button function
def US_Trades_Viz_Clean_Clicked(b):  
    with US_trades_viz_output_clean:
        
        # Clean output
        US_trades_viz_output.clear_output()

In [ ]:
##### US TRADES VISUALIZATION ON BUTTON CLIKED #####

# On button click
US_trades_viz_button.on_click(US_Trades_Viz_Clicked)
US_trades_viz_clean.on_click(US_Trades_Viz_Clean_Clicked)

In [ ]:
##### US FUTURTES RECEIPT CHECKBOX #####

# Create vector with checkbox data
data = ['Receipt Value', 'FX Description', 'FX Value', 'FX Rate', 'Comm', 'Exc Fee']

# Create checkbox receipt
US_futures_checkboxes = [widgets.Checkbox(value=True, indent=False, description=label) for label in data]

# Create checkbox financial horizontal box
US_futures_hbox = widgets.HBox(children=US_futures_checkboxes, layout=Layout(width='900px'))

In [ ]:
##### US FUTUES RECEIPT CUMSUM CHECKBOX #####

# Create vector with cumsum
data = ['Receipt Value Cum Sum', 'FX Value Cum Sum', 'Total Cum Sum', 'Settlement Cum Sum']

# Create checkbox cumsum
US_futures_checkboxes_cumsum = [widgets.Checkbox(value=True, indent=False, description=label) for label in data]

# Create checkbox cumsum horizontal box
US_futures_hbox_cumsum = widgets.HBox(children=US_futures_checkboxes_cumsum, layout=Layout(width='700px'))

In [ ]:
##### US FUTURES RECEIPT BUTTONS #####

# Create button and output
US_futures_button = Button_Sumbmit()

# Create button output
US_futures_output = widgets.Output()

# Create button and output
US_futures_clean = Button_Clean()

# Create button output
US_futures_output_clean = widgets.Output()

# Unite buttons
US_futures_ui = widgets.HBox([US_futures_button, US_futures_clean])

In [ ]:
##### US FUTURES RECEIPT BUTTONS FUNCTIONS #####

# Create button clicked function
def US_Futures_Clicked(b):
    with US_futures_output:
        
        # Clean output
        clear_output()
        
        # Create receipt dataframe
        df = US_futures_receipt
        
        # Create selected_data list
        selected_data = []
        
        # Add fixed variables
        selected_data = selected_data + ['Date']
                    
        # Loop operation filter
        for i in range(0, len(US_futures_checkboxes)):
                if US_futures_checkboxes[i].value == True:
                    selected_data = selected_data + [US_futures_checkboxes[i].description]
                                       
        # Loop financial filter
        for j in range(0, len(US_futures_checkboxes_cumsum)):
                if US_futures_checkboxes_cumsum[j].value == True:
                    selected_data = selected_data + [US_futures_checkboxes_cumsum[j].description]
        
        # Filter selected data
        df = df[selected_data]
  
        # Transform into date type
        df['Date'] = pd.to_datetime(df['Date'], format='%d/%m/%Y')
        
        # Apply filter
        df = df[(df['Date'] >=  pd.Timestamp(begin_period.value)) & (df['Date'] <= pd.Timestamp(end_period.value))] 
        
        # Sort by date
        df = df.sort_values(by = 'Date')
        
        # Refine date
        df['Date'] = df['Date'].dt.strftime('%d/%m/%Y')
        
        # Reset index
        df = df.reset_index(drop =True)
        
        # Red column vector
        red_col = ['Receipt Value', 'FX Value', 'Receipt Value Cum Sum', 'FX Value Cum Sum', 'Settlement Cum Sum']
        
        # Create auxiliar variable
        aux = []
        
        # Loop to define red columns
        for i in range(0,len(red_col)):
            if(red_col[i] in df.columns):
                aux.append(red_col[i])
        if aux == []:
            
            # Print result (without red columns)
            display(df.style.set_precision(2))
            
        else:
            
            # Print result (with red columns)
            display(df.style.applymap(color_negative_red, subset=pd.IndexSlice[:, aux]).set_precision(2))
        
# Create button clicked function
def US_Futures_Clean_Clicked(b):
    with US_futures_output_clean:
        
        # Clear output_receipt
        US_futures_output.clear_output()

In [ ]:
##### US FUTURES RECEIPT ON BUTTON CLIKED #####

# On button click
US_futures_button.on_click(US_Futures_Clicked)
US_futures_clean.on_click(US_Futures_Clean_Clicked)

In [ ]:
##### US EQUITIES RECEIPT CHECKBOX #####

# Create vector with checkbox data
data = ['Receipt Value', 'FX Value', 'FX Description', 'MtoM P&L', 'Comm', 'Exc Fee']

# Create checkbox receipt
US_equities_checkboxes = [widgets.Checkbox(value=True, indent=False, description=label) for label in data]

# Create checkbox financial horizontal box
US_equities_hbox = widgets.HBox(children=US_equities_checkboxes, layout=Layout(width='800px'))

In [ ]:
##### US EQUITES RECEIPT CUMSUM CHECKBOX #####

# Create vector with cumsum
data = ['Receipt Value Cum Sum', 'Settlement Cum Sum']

# Create checkbox cumsum
US_equities_checkboxes_cumsum = [widgets.Checkbox(value=True, indent=False, description=label) for label in data]

# Create checkbox cumsum horizontal box
US_equities_hbox_cumsum = widgets.HBox(children=US_equities_checkboxes_cumsum, layout=Layout(width='500px'))

In [ ]:
##### US EQUITIES RECEIPT BUTTONS #####

# Create button and output
US_equities_button = Button_Sumbmit()

# Create button output
US_equities_output = widgets.Output()

# Create button and output
US_equities_clean = Button_Clean()

# Create button output
US_equities_output_clean = widgets.Output()

# Unite buttons
US_equities_ui = widgets.HBox([US_equities_button, US_equities_clean])

In [ ]:
##### US EQUITIES RECEIPT BUTTONS FUNCTIONS #####

# Create button clicked function
def US_Equities_Clicked(b):
    with US_equities_output:
        
        # Clean output
        clear_output()
        
        # Create receipt dataframe
        df = US_equities_receipt
        
        # Create selected_data list
        selected_data = []
        
        # Add fixed variables
        selected_data = selected_data + ['Date']
                    
        # Loop operation filter
        for i in range(0, len(US_equities_checkboxes)):
                if US_equities_checkboxes[i].value == True:
                    selected_data = selected_data + [US_equities_checkboxes[i].description]
                                       
        # Loop financial filter
        for j in range(0, len(US_equities_checkboxes_cumsum)):
                if US_equities_checkboxes_cumsum[j].value == True:
                    selected_data = selected_data + [US_equities_checkboxes_cumsum[j].description]
        
        # Filter selected data
        df = df[selected_data]
  
        # Transform into date type
        df['Date'] = pd.to_datetime(df['Date'], format='%d/%m/%Y')
        
        # Apply filter
        df = df[(df['Date'] >=  pd.Timestamp(begin_period.value)) & (df['Date'] <= pd.Timestamp(end_period.value))] 
        
        # Sort by date
        df = df.sort_values(by = 'Date')
        
        # Refine date
        df['Date'] = df['Date'].dt.strftime('%d/%m/%Y')
        
        # Reset index
        df = df.reset_index(drop =True)
        
        # Red column vector
        red_col = ['Receipt Value', 'MtoM P&L', 'Receipt Value Cum Sum', 'Settlement Cum Sum']
        
        # Create auxiliar variable
        aux = []
        
        # Loop to define red columns
        for i in range(0,len(red_col)):
            if(red_col[i] in df.columns):
                aux.append(red_col[i])
        if aux == []:
            
            # Print result (without red columns)
            display(df.style.set_precision(2))
            
        else:
            
            # Print result (with red columns)
            display(df.style.applymap(color_negative_red, subset=pd.IndexSlice[:, aux]).set_precision(2))
        
# Create button clicked function
def US_Equities_Clean_Clicked(b):
    with US_equities_output_clean:
        
        # Clear output_receipt
        US_equities_output.clear_output()

In [ ]:
##### US EQUITIES RECEIPT ON BUTTON CLIKED #####

# On button click
US_equities_button.on_click(US_Equities_Clicked)
US_equities_clean.on_click(US_Equities_Clean_Clicked)

In [ ]:
##### ALL TRADES PERIOD SELECT DROPDOWN MENU #####

# Function to select period
def Period_Select(period):
    
    if (period == 'Daily'):
        
        # Return output
        return(all_trades)
    
    elif (period == 'Monthly'):
        
        # Get all trades dataframe
        df = all_trades
        
        # Transform into date type
        df['Date'] = pd.to_datetime(df['Date'], format='%d/%m/%Y')
        
        # Group all trades data monthly
        dolar_mean = df.resample('M', on='Date' ,closed="right", label="right").mean().reset_index()['Dolar Ref']
        df = df.resample('M', on='Date' ,closed="right", label="right").sum().reset_index()
        
        # Set cumulative sum
        df['P&L Cum Sum'] = df['Total P&L'].cumsum()
        df['Return Cum Sum'] = df['Return in %'].cumsum()
        df['Dolar Ref'] = dolar_mean
        
        # Refine date
        df['Date'] = df['Date'].dt.strftime('%b %Y')
        
        # Return output
        return(df)
    
    else:
        
        # Get all trades dataframe
        df = all_trades
        
        # Transform into date type
        df['Date'] = pd.to_datetime(df['Date'], format='%d/%m/%Y')
        
        # Group all trades data quarterly
        dolar_mean = df.resample('3M', on='Date' ,closed="right", label="right").mean().reset_index()['Dolar Ref']
        output = df.resample('3M', on='Date' ,closed="right", label="right").sum().reset_index()
        
        # Get final date
        output['Date 2'] = df.resample('3M', on='Date' ,closed="left", label="right").sum().reset_index()['Date']
        
        # Set cumulative sum
        output['P&L Cum Sum'] = output['Total P&L'].cumsum()
        output['Return Cum Sum'] = output['Return in %'].cumsum()
        output['Dolar Ref'] = dolar_mean
        
        # Clean dates format
        output['Date'] = output['Date'].dt.strftime('%b %y')
        output['Date 2'] = output['Date 2'].dt.strftime('%b %y')
        
        # Create final date column
        output['Date Final'] = output['Date'] + ' to ' + output['Date 2']
        
        # Fill missing values
        output['Date Final'] = output['Date Final'].fillna(output['Date'].tail(1))
        
        # Remove Date columns
        del(output['Date 2'])
        del(output['Date'])
        
        # Rename column
        output.rename(columns = {'Date Final' : 'Date'}, inplace = True)
        
        # Return output
        return(output)

In [ ]:
##### ALL TRADES DROPDOWNMENU #####
all_trades_period_dropdown = interactive(Period_Select,
                                 period = widgets.Dropdown(
                                 options= ['Daily','Monthly','Quarterly'],
                                 description= "Period:")
                               )

In [ ]:
##### ALL TRADES BR CHECKBOX #####

# Create vector with checkbox data
data = ['BR Activity', 'BR Futures', 'Dolar Ref', 'BR Futures P&L']

# Create checkbox receipt
all_trades_BR_checkboxes = [widgets.Checkbox(value= True, indent= False, description= label) for label in data]

# Create checkbox financial horizontal box
all_trades_BR_hbox = widgets.HBox(children= all_trades_BR_checkboxes, layout= Layout(width= '700px'))

In [ ]:
##### ALL TRADES US CHECKBOX #####

# Create vector with checkbox data
data = ['US Activity', 'US Futures', 'US Futures P&L', 'US Equities P&L']

# Create checkbox receipt
all_trades_US_checkboxes = [widgets.Checkbox(value= True, indent= False, description= label) for label in data]

# Create checkbox financial horizontal box
all_trades_US_hbox = widgets.HBox(children= all_trades_US_checkboxes, layout= Layout(width= '700px'))

In [ ]:
##### ALL TARDES CUMSUM CHECKBOX #####

# Create vector with cumsum
data = ['Futures P&L', 'Total P&L', 'Return in %', 'Total P&L Cum Sum', 'Return Cum Sum']

# Create checkbox cumsum
all_trades_total_checkboxes = [widgets.Checkbox(value=True, indent=False, description=label) for label in data]

# Create checkbox cumsum horizontal box
all_trades_total_hbox = widgets.HBox(children=all_trades_total_checkboxes, layout=Layout(width='800px'))

In [ ]:
##### ALL TRADES BUTTONS #####

# Create button and output
all_trades_button = Button_Sumbmit()

# Create button output
all_trades_output = widgets.Output()

# Create button and output
all_trades_clean = Button_Clean()

# Create button output
all_trades_output_clean = widgets.Output()

# Unite buttons
all_trades_ui = widgets.HBox([all_trades_button, all_trades_clean])

In [ ]:
##### ALL TRADES BUTTONS FUNCTIONS #####

# Create button clicked function
def All_Trades_Clicked(b):
    with all_trades_output:
        
        # Clean output
        clear_output()
        
        # Create receipt dataframe
        df = all_trades_period_dropdown.result
        
        # Create selected_data list
        selected_data = []
        
        # Add fixed variables
        selected_data = selected_data + ['Date']
        
        # Loop BR
        for i in range(0, len(all_trades_BR_checkboxes)):
                if all_trades_BR_checkboxes[i].value == True:
                    selected_data = selected_data + [all_trades_BR_checkboxes[i].description]
                    
        # Loop US
        for i in range(0, len(all_trades_US_checkboxes)):
                if all_trades_US_checkboxes[i].value == True:
                    selected_data = selected_data + [all_trades_US_checkboxes[i].description]
                    
        # Loop total
        for j in range(0, len(all_trades_total_checkboxes)):
                if all_trades_total_checkboxes[j].value == True:
                    selected_data = selected_data + [all_trades_total_checkboxes[j].description]
        
        # Filter selected data
        df = df[selected_data]
        
        if (all_trades_period_dropdown.result['Date'].dtypes == '<M8[ns]'):
            
            # Transform into date type
            df['Date'] = pd.to_datetime(df['Date'], format='%d/%m/%Y')
        
            # Apply filter
            df = df[(df['Date'] >=  pd.Timestamp(begin_period.value)) & (df['Date'] <= pd.Timestamp(end_period.value))] 
        
            # Sort by date
            df = df.sort_values(by = 'Date')
                           
            # Refine date
            df['Date'] = df['Date'].dt.strftime('%d/%m/%Y')
        
        elif (len(all_trades_period_dropdown.result['Date'][0]) == 8):
            
            # Set date format            
            df['Date'] = pd.to_datetime(df['Date'], format='%b %Y')
            
            # Apply filter            
            df = df[(df['Date'] >= dt.datetime.strptime(str(begin_period.value)[0:10], '%Y-%m-%d').strftime('%b %Y')) & 
                    (df['Date'] <= dt.datetime.strptime(str(end_period.value)[0:10], '%Y-%m-%d').strftime('%b %Y'))] 
            
             # Sort by date
            df = df.sort_values(by = 'Date')
            
            # Refine date
            df['Date'] = df['Date'].dt.strftime('%b %Y')
                                                
        else:
            
        #elif (len(all_trades_dropdown.result['Date'][0]) == 16):
            
            pass            
        
        # Reset index
        df = df.reset_index(drop =True)
        
        # Red column vector
        red_col = df.columns.drop('Date')
        
        # Create auxiliar variable
        aux = []
        
        # Loop to define red columns
        for i in range(0,len(red_col)):
            if(red_col[i] in df.columns):
                aux.append(red_col[i])
        if aux == []:
            
            # Print result (without red columns)
            display(df)
            
        else:
            
            # Print result (with red columns)
            display(df.style.applymap(color_negative_red, subset=pd.IndexSlice[:, aux]).set_precision(2))
        
# Create button clicked function
def All_Trades_Clean_Clicked(b):
    with all_trades_output_clean:
        
        # Clear output_receipt
        all_trades_output.clear_output()

In [ ]:
##### ALL TRADES ON BUTTON CLIKED #####

# On button click
all_trades_button.on_click(All_Trades_Clicked)
all_trades_clean.on_click(All_Trades_Clean_Clicked)

In [ ]:
##### ALL TRADES P&L DROPDOWN MENU #####

# Function to select period
def Prof_and_Loss_Select(prof_and_loss):

    #print(prof_and_loss)
 
    return()

In [ ]:
###### ALL TRADES DROPDOWN MENUS #####
prof_and_loss_dropdown = interactive(Prof_and_Loss_Select,
                                 prof_and_loss = widgets.Dropdown(
                                 options= ['Total P&L','BR Futures P&L','US Futures P&L', 'US Equities P&L'],
                                 description= "P&L:")
                               )

In [ ]:
##### ALL TRADES PERIOD SELECT DROPDOWN MENU #####

# Function to select period
def Year_Select(year):
    
    #print(month)
 
    return()

In [ ]:
###### ALL TRADES DROPDOWN MENUS #####
all_trades_year_dropdown = interactive(Year_Select,
                                 year = widgets.Dropdown(
                                 options= ['All','2020','2021'],
                                 description= "Year:")
                               )

In [ ]:
##### ALL TRADES PERIOD SELECT DROPDOWN MENU #####

# Function to select period
def Month_Select(month):
    
    #print(month)
 
    return()

In [ ]:
###### ALL TRADES DROPDOWN MENUS #####
all_trades_month_dropdown = interactive(Month_Select,
                                 month = widgets.Dropdown(
                                 options= ['All','Jan','Feb','Mar','Abr','May','Aug','Sep','Oct','Nov','Dec'],
                                 description= "Month:")
                               )

In [ ]:
###### ALL TRADES VISUALIZATION BUTTONS #####

# Create button and output
all_trades_viz_button = Button_Sumbmit()

# Create button output
all_trades_viz_output = widgets.Output()

# Create button and output
all_trades_viz_clean = Button_Clean()

# Create button output
all_trades_viz_output_clean = widgets.Output()

# Unite buttons
all_trades_viz_ui = widgets.HBox([all_trades_viz_button, all_trades_viz_clean])

In [ ]:
##### ALL TRADES VISUALIZATION BUTTONS FUNCTIONS #####

# Create button clicked function
def All_Trades_Viz_Clicked(b):  
    with all_trades_viz_output:
        
        # Clean output
        clear_output()
                    
        # Create trade dataframe
        df = all_trades
        
        # Transform into date type
        df['Date'] = pd.to_datetime(df['Date'], format='%d/%m/%Y')
        
        # Group all trades data monthly
        df = df.resample('M', on='Date' ,closed="right", label="right").sum().reset_index()
        
        # Create cumulative sum
        df['Cum Sum'] = df['Total P&L'].cumsum()
        
        # Refine date
        df['Date'] = df['Date'].dt.strftime('%b %Y')
        
        # Set date format            
        df['Date'] = pd.to_datetime(df['Date'], format='%b %Y')
        
        # Apply filter            
        df = df[(df['Date'] >= dt.datetime.strptime(str(begin_period.value)[0:10], '%Y-%m-%d').strftime('%b %Y')) & 
                (df['Date'] <= dt.datetime.strptime(str(end_period.value)[0:10], '%Y-%m-%d').strftime('%b %Y'))] 
            
#         # Sort by date
#         df = df.sort_values(by = 'Date')
            
#         # Refine date
#         df['Date'] = df['Date'].dt.strftime('%b %Y')

#         # Transform into date type
#         df['Date'] = pd.to_datetime(df['Date'], format='%d/%m/%Y')
        
        fig = px.bar(df, 
                    x="Date", 
                    y="Total P&L",  
                   text= df['Return in %'].apply(lambda x: '{0:1.2f}%'.format(x)), 
                    title='The plot shows trade ')
    
        fig.update_layout(uniformtext_minsize=10, uniformtext_mode='hide')
        
        fig.add_trace(go.Scatter(mode= "lines", x= df["Date"], y= df["Cum Sum"], name= "Cumulative Sum"))

        # Plot graph
        fig.show()
        
# Create clean button function
def All_Trades_Viz_Clean_Clicked(b):  
    with all_trades_viz_output_clean:
        
        # Clean output
        all_trades_viz_output.clear_output()

In [ ]:
##### US TRADES VISUALIZATION ON BUTTON CLIKED #####

# On button click
all_trades_viz_button.on_click(All_Trades_Viz_Clicked)
all_trades_viz_clean.on_click(All_Trades_Viz_Clean_Clicked)

This report is **interactive**. Therefore, you can make multiple analysis using filters to selects specific data to examine. The report can display financial variables results of the trades and their receipts.

The report uses the following references:

* P&L = Profit and Loss
* MtoM = Market to Market
* Comm = Commisions
* Exc Fee = Exchange Fee
* Reg Fee = Register Fee
* Px Close = Close Price
* Cum Sum = Cumulative Sum

To iniciate the analysis, select the time range you want to exam. After that, use the tabs to choose which trades you want to review.

### Select Time Range:



In [ ]:
# Display date range filter
display(begin_period, end_period)

In [ ]:
##### BRAZIL TRADES ANALYSIS #####

# Create analysis
BR_trade_analysis = widgets.VBox([
    
    # Title
    widgets.HTML('<center> <h1> Brazil Trades Data Analysis </h1> </center>'),
    
    # Trades table
    widgets.HTML('<i> <h1> Trades Table </h1> </i>'),
    widgets.HTML('The data shows financial operations and indicators results in <b> BR Real (R$) </b>, calculated for the Brazilian trades using the financial operations displayed.'),
    widgets.HTML('The table can show results from the following trades:'),
    widgets.HTML('&emsp; <i> DOL </i> - Comercial Dolar Futures'),
    widgets.HTML('&emsp; <i> WDO </i> - Mini Comercial Dolar Futures'),
    widgets.HTML('&emsp; <i> IND </i> - Ibovespa Futures'),
    widgets.HTML('&emsp; <i> WIN </i> - Mini Ibovespa Futures'),
    widgets.HTML('&emsp; <i> ISP </i> - S&P 500 Futures'),
    widgets.HTML('&emsp; <i> WSP </i> - Micro S&P 500 Futures'),
    widgets.HTML('<h3> Select Trade: </h3>'), 
    BR_trade_dropdown,
    widgets.HTML('<h3> Select Operation: </h3>'), 
    BR_hbox_operation,
    widgets.HTML('<h3> Select Indicator: </h3>'), 
    BR_hbox_financial,
    widgets.HTML('<br>'), 
    BR_trades_ui,
    widgets.HTML('<br>'), 
    BR_trades_output,
    BR_trades_output_clean,
    
    # Rolagem table
    widgets.HTML('<i> <h1> Rolagem Table </h1> </i>'),
    widgets.HTML('The table shows the Brazilian trades <i> rolagem </i> operation in <b> BR Real (R$) </b> for all trades.'),
    widgets.HTML('<h3> Select Rolagem: </h3>'), 
    BR_rolagem_dropdown,
    widgets.HTML('<h3> Select Operation: </h3>'), 
    BR_rolagem_hbox_operation,
    widgets.HTML('<h3> Select Indicator: </h3>'), 
    BR_rolagem_hbox_financial,
    widgets.HTML('<br>'), 
    BR_rolagem_ui,
    widgets.HTML('<br>'), 
    BR_rolagem_output,
    BR_rolagem_output_clean,
    
    # Add horizontal line
    widgets.HTML('<hr>'), 
    
    # Visualization
    widgets.HTML('<h2> Trades Data Visualization </h2>'),
    BR_trades_viz_ui,
    widgets.HTML('<br>'), 
    BR_trades_viz_output,
    BR_trades_viz_output_clean
])

In [ ]:
##### BRAZIL RECEIPT ANALYSIS #####

# Create analysis
BR_receipt_analysis = widgets.VBox([
    
    # Title
    widgets.HTML('<center> <h1> Brazil Receipt Data Analysis </h1> </center>'),
    
    # BR Receipt Table 
    widgets.HTML('<i> <h1> Receipt Table </h1> </i>'),
    widgets.HTML('The data shows the Brazilian trades recipt values in <b> BR Real (R$) </b>. The receipt values displayed contains the sum of all trades results.'),
    widgets.HTML('<h3> Select Operation: </h3>'), 
    BR_receipt_hbox,
    widgets.HTML('<h3> Select Cumulative Sum: </h3>'), 
    BR_receipt_hbox_cumsum,
    widgets.HTML('<br>'), 
    BR_receipt_ui,
    widgets.HTML('<br>'),
    BR_receipt_output,
    BR_receipt_output_clean, 
    
    # Add horizontal line
    widgets.HTML('<hr>'), 
    
    #### Visualization ####
    widgets.HTML('<h1> Receipt Visualization </h1>')
])

In [ ]:
##### US TRADES ANALYSIS #####

# Create analysis
US_trade_analysis = widgets.VBox([
            
        # Title
        widgets.HTML('<center> <h1> US Trades Data Analysis </h1> </center>'),
    
        # Futures Table
        widgets.HTML('<i> <h1> Futures Table </h1> </i>'),
        widgets.HTML('The data shows financial operations and indicators results in <b> US Dolar (US$) </b> calculated for the United States of America trades using the financial operations displayed.'),
        widgets.HTML('The <i> futures </i> table can show results from the following trades:'),
        widgets.HTML('&emsp; <i> 6L </i> - Brazilian Real Futures'),
        widgets.HTML('&emsp; <i> ES </i> - Mini S&P 500 Futures'),
        widgets.HTML('&emsp; <i> MES </i> - Micro and Mini S&P 500 Futures'),
        widgets.HTML('<h3> Select Trade: </h3>'), 
        US_futures_dropdown,
        widgets.HTML('<h3> Select Operation: </h3>'),
        US_futures_trades_hbox_operation,
        widgets.HTML('<h3> Select Indicator: </h3>'), 
        US_futures_trades_hbox_financial,
        widgets.HTML('<br>'), 
        US_futures_trades_ui,
        widgets.HTML('<br>'), 
        US_futures_trades_output,
        US_futures_trades_output_clean,
        
        # Equities Table
        widgets.HTML('<i> <h1> Equities Table </h1> </i>'),
        widgets.HTML('The <i> equities </i> table can show results from the following trades:'),
        widgets.HTML('&emsp; <i> MS </i> - Morgan Stanley Equities'),
        widgets.HTML('&emsp; <i> PBR </i> - Petrobras Equities'),
        widgets.HTML('&emsp; <i> SOXX </i> - iShares PHLX Semiconductor Equities'),
        widgets.HTML('<h3> Select Trade: </h3>'), 
        US_equities_dropdown,
        widgets.HTML('<h3> Select Operation: </h3>'),
        US_equities_trades_hbox_operation,
        widgets.HTML('<h3> Select Indicator: </h3>'), 
        US_equities_trades_hbox_financial,
        widgets.HTML('<br>'), 
        US_equities_trades_ui,
        widgets.HTML('<br>'), 
        US_equities_trades_output,
        US_equities_trades_output_clean, 
        
        # Add horizontal line
        widgets.HTML('<hr>'), 
    
        ##### Visualization ####
    
        widgets.HTML('<h1> Trades Visualization </h1>'),
        US_trades_viz_ui,
        widgets.HTML('<br>'), 
        US_trades_viz_output,
        US_trades_viz_output_clean
])

In [ ]:
##### US RECEIPT ANALYSIS #####

# Create analysis
US_receipt_analysis = widgets.VBox([
    
    # Title
    widgets.HTML('<center> <h1> US Receipt Data Analysis </h1> </center>'),

    # Futures Table
    widgets.HTML('<i> <h1> Futures Trades </h1> </i>'),
    widgets.HTML('The table shows the US trades recipt values in <b> US Dolar (US$) </b>. The receipt values displayed contains the sum of all <i> futures </i> trades results.'),
    widgets.HTML('<h3> Select Operation: </h3>'), 
    US_futures_hbox,
    widgets.HTML('<h3> Select Cumulative Sum: </h3>'), 
    US_futures_hbox_cumsum,
    widgets.HTML('<br>'), 
    US_futures_ui,
    widgets.HTML('<br>'), 
    US_futures_output,
    US_futures_output_clean,
    
    # Equities Table
    widgets.HTML('<i> <h1> Equities Trades </h1> </i>'),
    widgets.HTML('The table shows the US trades recipt values in <b> US Dolar (US$) </b>. The receipt values displayed contains the sum of all <i> equities </i> trades results.'),
    widgets.HTML('<h3> Select Operation: </h3>'), 
    US_equities_hbox,
    widgets.HTML('<h3> Select Cumulative Sum: </h3>'), 
    US_equities_hbox_cumsum,
    widgets.HTML('<br>'),
    US_equities_ui,
    widgets.HTML('<br>'), 
    US_equities_output,
    US_equities_output_clean,
    
    # Add horizontal line
    widgets.HTML('<hr>'), 
    
    # Visualizations
    widgets.HTML('<h2> Receipt Data Visualization </h2>')
])

In [ ]:
##### ALL TRADES ANALYSIS #####

# Create analysis
All_trades_analysis = widgets.VBox([
    
    # Title
    widgets.HTML('<center> <h1> Total Receipt Data Analysis </h1> </center>'),
    
    # Table
    widgets.HTML('<i> <h1> Receipt Table </h1> </i>'),
    widgets.HTML('The data shows the United States of American and Brazilian trades recipt values in <b> US Dolar (US$) </b>. Beyond that, the table also shows the total receipt value and it´s return.'),
    widgets.HTML('<h3> Select Period: </h3>'), 
    all_trades_period_dropdown,
    widgets.HTML('<h3> Select BR Results: </h3>'), 
    all_trades_BR_hbox,
    widgets.HTML('<h3> Select US Results: </h3>'), 
    all_trades_US_hbox,
    widgets.HTML('<h3> Select Total Results: </h3>'), 
    all_trades_total_hbox,
    widgets.HTML('<br>'), 
    all_trades_ui,
    widgets.HTML('<br>'), 
    all_trades_output,
    all_trades_output_clean, 
    
    # Add horizontal line
    widgets.HTML('<hr>'), 
    
    # Visualization
    widgets.HTML('<i> <h1> Receipt Visualization </h1> </i>'),
    widgets.HTML('<h3> Select Profit and Loss: </h3>'), 
    prof_and_loss_dropdown,
    widgets.HTML('<h3> Select Year: </h3>'), 
    all_trades_year_dropdown,
    widgets.HTML('<h3> Select Month: </h3>'), 
    all_trades_month_dropdown,
    all_trades_viz_ui,
    widgets.HTML('<br>'), 
    all_trades_viz_output,
    all_trades_viz_output_clean
])

In [ ]:
##### DISPLAY TABS #####

# Create nested tab
tab_nest = widgets.Tab()

# Set each tab analysis
tab_nest.children = [BR_trade_analysis, BR_receipt_analysis, US_trade_analysis, US_receipt_analysis, All_trades_analysis]

# Set tab title
tab_nest.set_title(0, 'Brazil Trades')
tab_nest.set_title(1, 'Brazil Recipt')
tab_nest.set_title(2, 'US Trades')
tab_nest.set_title(3, 'US Receipt')
tab_nest.set_title(4, 'Total Receipt')

# Plot tab
tab_nest